In [1]:
import pandas as pd
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import MinMaxScaler,Normalizer,StandardScaler,MaxAbsScaler, LabelEncoder

#import graphviz

/home/marcosfabricio/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#Transformando a coluna de sexo de testo pra numero e considerando ela como dado categorico
df = pd.read_csv("abalone_sex_bin.csv")
df['sex'] = LabelEncoder().fit_transform(df['sex'].tolist())
df['sex'] = df['sex'].astype('category')
# df = df.drop(columns = "sex")

In [3]:
# Preprocessamento da base
preps = [MinMaxScaler,Normalizer,StandardScaler,MaxAbsScaler]
# Modelos a serem testados
models = [SVC,LogisticRegression,MLPClassifier,RandomForestClassifier,DecisionTreeClassifier]
# Pipeline para testar todos os modelos com todos os preprocessamento
pipes = [make_pipeline(prepo(),model()) for model in models for prepo in preps]

In [4]:
results = []
for pipe in pipes:
    res = np.median(cross_validate(pipe,df.drop(columns="type"),df["type"],scoring="accuracy",cv=10)["test_score"])
    results.append(np.append(np.array(pipe.steps)[:,0],res))

In [5]:
df.dtypes

sex               category
length             float64
diameter           float64
height             float64
whole_weight       float64
shucked_weight     float64
viscera_weight     float64
shell_weight       float64
type                 int64
dtype: object

In [6]:
pd.DataFrame(results,columns=["Preprocessing","Model","Median-Accuracy"]).sort_values(by="Median-Accuracy",ascending=False)

,Preprocessing,Model,Median-Accuracy
10,standardscaler,mlpclassifier,0.660830
11,maxabsscaler,mlpclassifier,0.657099
6,standardscaler,logisticregression,0.654952
2,standardscaler,svc,0.653904
9,normalizer,mlpclassifier,0.646412
8,minmaxscaler,mlpclassifier,0.640002
4,minmaxscaler,logisticregression,0.638404
14,standardscaler,randomforestclassifier,0.637380
7,maxabsscaler,logisticregression,0.636807
13,normalizer,randomforestclassifier,0.634615


# Pegamos o melhor modelo e preprocessamento, para testar no GridSearch

In [7]:
# Pipeline para o grid search
pipe = make_pipeline(MinMaxScaler(feature_range=(0.1,0.9)),MLPClassifier())
# Dicionario de parametros a serem testados pelo grid search
logparameters = {'logisticregression__penalty':['l2'], 'logisticregression__solver':('newton-cg', 'lbfgs', 'sag', 'saga'), 'logisticregression__C': (np.arange(10,100,10)), 'logisticregression__multi_class':['multinomial'], 'logisticregression__max_iter':[1000]}
mlpparameters = {'activation':['tanh'], 'solver': ['lbfgs'], 'max_iter': [1000], 'alpha': [1], 'hidden_layer_sizes':[(16,)]}

In [8]:
# GridSearch com cros validation, testa o modelo com todas as combinações de parametros passadas no dicionario,
# e classifica a melhor de acordo com uma metrica que escolhermos, nesse caso a acuracia.
clf = GridSearchCV(MLPClassifier(),mlpparameters,scoring="accuracy", cv=10, n_jobs=-1)

In [9]:
%%time
clf.fit(df.drop(columns="type"),df["type"])

CPU times: user 4.06 s, sys: 112 ms, total: 4.17 s
Wall time: 18.7 s


GridSearchCV(cv=10, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'activation': ['tanh'], 'solver': ['lbfgs'], 'max_iter': [1000], 'alpha': [1], 'hidden_layer_sizes': [(16,)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

# Seleção de atributos recursivamente
- selecionamos a melhor combinação de hiperparametros do modelo com o grid search
- aplicamos a seleção de atributos nesse modelo

In [10]:
clf.best_estimator_

MLPClassifier(activation='tanh', alpha=1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(16,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [11]:
#seletor = RFECV(log, cv=10, scoring='accuracy')

In [12]:
#seletor.fit(df.drop(columns="type"),df["type"])

In [13]:
#Atributos selecionados
#df.drop(columns="type").columns[seletor.get_support()]

In [14]:
# plt.figure()
# plt.xlabel("Number of features selected")
# plt.ylabel("Cross validation score (accuracy)")
# plt.plot(range(1, len(seletor.grid_scores_) + 1), seletor.grid_scores_)
# plt.show()

In [15]:
# Gerando vetor resposta pra enviar ao servidor

In [16]:
#mlp = MLPClassifier(activation='tanh', alpha=0.30000000000000004,hidden_layer_sizes=(16,), learning_rate='invscaling',learning_rate_init=0.1, max_iter=100, solver='lbfgs')

In [17]:
#bla = cross_validate(mlp,df.drop(columns="type"),df["type"],scoring="accuracy",cv=10, n_jobs=-1)

In [18]:
# df = pd.read_csv("abalone_dataset.csv")
# df.head()

# df.dtypes

# df = df.drop(columns="sex")
# type = df["type"]

#scaler = MinMaxScaler(feature_range=(0.1,0.9))
# dfscaled = scaler.fit_transform(df.drop(columns="type"), df["type"])
# dfscaled = pd.DataFrame(dfscaled, columns=df.columns[:-1])

# dfscaled = pd.merge(dfscaled,pd.DataFrame(type),right_index=True,left_index=True)

In [19]:
#mlp.fit(dfscaled.drop(columns="type"),dfscaled["type"])
teste = pd.read_csv("abalone_app.csv")
teste['sex'] = LabelEncoder().fit_transform(teste['sex'].tolist())
teste['sex'] = teste['sex'].astype('category')
# teste = teste.drop(columns="sex")
#testescaled = scaler.fit_transform(teste)
# testescaled = pd.DataFrame(testescaled,columns=teste.columns)
pd.Series(clf.predict(teste)).to_csv("respostas.csv",index=False)
bla = clf.predict(teste)

In [20]:
y_pred = pd.read_csv("respostas.csv").values

In [21]:
import requests

print('\n - Lendo o arquivo com o dataset sobre abalone')

# Enviando previsões realizadas com o modelo para o servidor
URL = "https://aydanomachado.com/mlclass/03_Validation.php"

#TODO Substituir pela sua chave aqui
DEV_KEY = 'Ponte de Safena'

# json para ser enviado para o servidor
y_pred1 = pd.Series(bla)
data = {'dev_key':DEV_KEY,
        'predictions':y_pred1.to_json(orient='values')}

# Enviando requisição e salvando o objeto resposta
r = requests.post(url = URL, data = data)

# Extraindo e imprimindo o texto da resposta
pastebin_url = r.text
print(" - Resposta do servidor:\n", r.text, "\n")


 - Lendo o arquivo com o dataset sobre abalone
 - Resposta do servidor:
 {"status":"success","dev_key":"Ponte de Safena","accuracy":0.6641148325358852,"old_accuracy":0.60956937799043} 

